<a href="https://colab.research.google.com/github/Matancoo/IML.HUJI/blob/main/challenge/DATA_CHALLENGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import pickle

# from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping


import torch
import torchvision
import torchvision.transforms as transforms
import random 
from torchvision import transforms, datasets
from torch.utils.data import DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

In [98]:



def load_data(filename: str):
    """
    Load Agoda booking cancellation dataset
    Parameters
    ----------
    filename: str
        Path to house prices dataset
    Returns
    -------
    Design matrix and response vector in either of the following formats:
    1) Single dataframe with last column representing the response
    2) Tuple of pandas.DataFrame and Series
    3) Tuple of ndarray of shape (n_samples, n_features) and ndarray of shape (n_samples,)
    """
    # Data Preprocessing
    data = pd.read_csv(filename)#.drop_duplicates()
    data.dropna(subset = ['origin_country_code','origin_country_code'], inplace = True)
    data['request_nonesmoke'].replace(to_replace=np.nan, value = 0.0, inplace= True)
    data['request_latecheckin'].replace(to_replace=np.nan, value=0.0, inplace = True)
    data['request_highfloor'].replace(to_replace=np.nan, value=0.0, inplace = True)
    data['request_largebed'].replace(to_replace=np.nan, value=0.0, inplace = True)
    data['request_twinbeds'].replace(to_replace=np.nan, value=0.0, inplace = True)
    data['request_airport'].replace(to_replace=np.nan, value=0.0, inplace = True)
    data['request_earlycheckin'].replace(to_replace=np.nan, value=0.0, inplace = True)



    # Changing dates to datetime type: 
    data['cancellation_datetime'] = pd.to_datetime(data['cancellation_datetime'])
    data['booking_datetime'] = pd.to_datetime(data['booking_datetime'])
    data['checkout_date'] = pd.to_datetime(data['checkout_date'])
    data['hotel_live_date'] = pd.to_datetime(data['hotel_live_date'])
    data['checkin_date'] = pd.to_datetime(data['checkin_date'])


    #feature engineering
    df.cancellation_datetime = df['cancellation_datetime'].apply(lambda x: x.date())
    start_date = pd.to_datetime('2018-12-07')
    end_date = pd.to_datetime('2018-12-13')
    # df['cancellation_datetime'] = (df.cancellation_datetime <= end_date) &  (df.cancellation_datetime > start_date)

    #whether a person cancelled or not.
    data['is_cancelled'] = (data.cancellation_datetime !=np.nan)




    # Dropping samples that have a checkin-date after 2018-12-13
    #Turning objects to one-hot vectors:
    # pd.get_dummies()
    
    labels = data.pop("cancellation_datetime")
    features = data





def evaluate_and_export(estimator, X: np.ndarray, filename: str):
    """
    Export to specified file the prediction results of given estimator on given testset.
    File saved is in csv format with a single column named 'predicted_values' and n_samples rows containing
    predicted values.
    Parameters
    ----------
    estimator: BaseEstimator or any object implementing predict() method as in BaseEstimator (for example sklearn)
        Fitted estimator to use for prediction
    X: ndarray of shape (n_samples, n_features)
        Test design matrix to predict its responses
    filename:
        path to store file at
    """
    pd.DataFrame(estimator.predict(X), columns=["predicted_values"]).to_csv(filename, index=False)




Data Preprocessing

In [99]:
from pandas.io.parsers.readers import read_csv
#loading data:
data, labels = load_data('https://raw.githubusercontent.com/Matancoo/IML.HUJI/main/datasets/agoda_cancellation_train.csv')



In [116]:
#check if data missing 
df = pd.read_csv('https://raw.githubusercontent.com/Matancoo/IML.HUJI/main/datasets/agoda_cancellation_train.csv')
data.isnull().sum()

x =labels.sort_values().reset_index()
x


,index,cancellation_datetime
0,27395,2017-08-11
1,10655,2017-08-12
2,40764,2017-08-23
3,24976,2017-09-03
4,52675,2017-09-03
...,...,...
58652,58653,NaT
58653,58655,NaT
58654,58656,NaT
58655,58657,NaT


In [73]:

# # returns a correlation matrix of each feature with all others.
# data = df.select_dtypes(include=np.number)  # select only numerical features
# corr_matrix = data.corr(method='pearson').loc[:, ['cancellation_datetime']].sort_values('cancellation_datetime')
# corr_target = abs(corr_matrix)
# corr_target



KeyError: ignored

In [113]:
data['checkin_date'].sort_values().head(20)

23962   2018-06-07
16951   2018-06-15
31122   2018-06-16
18350   2018-06-17
38215   2018-06-21
48650   2018-06-21
37615   2018-06-21
34807   2018-06-21
47352   2018-06-21
2497    2018-06-22
6505    2018-06-22
55629   2018-06-22
51175   2018-06-22
12480   2018-06-23
58459   2018-06-23
38119   2018-06-23
33131   2018-06-23
55683   2018-06-23
15091   2018-06-23
46389   2018-06-23
Name: checkin_date, dtype: datetime64[ns]

In [ ]:
# if __name__ == '__main__':
#     np.random.seed(0)

#     # Load data
#     df, cancellation_labels = load_data("../datasets/agoda_cancellation_train.csv")
#     train_X, train_y, test_X, test_y = split_train_test(df, cancellation_labels)

#     # Fit model over data
#     estimator = AgodaCancellationEstimator().fit(train_X, train_y)

#     # Store model predictions over test set
#     evaluate_and_export(estimator, test_X, "id1_id2_id3.csv")